In [93]:
import os, json,unstructured_client
from unstructured_client.models import operations, shared
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError
from pathlib import Path
from typing import List, Tuple
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()
client = unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)
def get_elements_from_file(file_path:str,strategy:shared.Strategy=shared.Strategy.FAST):
    with open(file_path, "rb") as f:          # pass bytes, not a file handle
        file_bytes = f.read()
            # print first 10 bytes
    filename=os.path.basename(file_path)
    req = operations.PartitionRequest(
        partition_parameters=shared.PartitionParameters(
            strategy=strategy,
            files=shared.Files(
                content=file_bytes,
                file_name=filename,
                
            ),
            
            languages=["heb", "eng"],
        )
    )

    client = unstructured_client.UnstructuredClient(
        api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
    )

    try:  
        res = client.general.partition(request=req)
        elements = res.elements                          # list of Element objects
        
        return elements
    except SDKError as e:                                # friendlier error handling
        print(f"{e.status_code}: {e.message}")

def list_all_files(
    start: str | Path,
    exts: Tuple[str, ...] = (".pdf", ".doc", ".docx"),
) -> List[str]:
    """
    Return a list of full (absolute) paths for every *PDF, DOC, or DOCX* file
    inside *start* and all its sub-directories.

    Parameters
    ----------
    start : str | Path
        The root folder to walk. Tilde (~) is expanded and the path is
        resolved so the result is always absolute.
    exts : tuple[str, ...], optional
        File-name suffixes to include (case-insensitive). Default picks
        '.pdf', '.doc', and '.docx'.

    Raises
    ------
    FileNotFoundError
        If *start* does not exist or is not a directory.
    """
    root = Path(start).expanduser().resolve()

    if not root.is_dir():
        raise FileNotFoundError(f"{root} is not an existing directory")

    # rglob('*') walks recursively; filter by suffix (case-insensitive)
    return [
        str(p)
        for p in root.rglob("*")
        if p.is_file() and p.suffix.lower() in exts
    ]
# def check_for_scanned_pdf(filename):
#     """
#     Check if the file is a scanned PDF by checking the first 10 bytes.
#     """
#     with open(filename

In [79]:





pathes=list_all_files("docs")


In [80]:
for path in pathes:
    print(path)
    

/Users/matansharon/python/AI/unsructered/docs/ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf
/Users/matansharon/python/AI/unsructered/docs/GaLore- Memory-Efficient LLM Training by Gradient Low-Rank Projection.pdf
/Users/matansharon/python/AI/unsructered/docs/2403.06634.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP - Bonded Manifolds - Rev B_scanned for TF.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/Risk Management Plan - Bonded Manifolds.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP - Bonded Manufolds - Rev A.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMF - bonded manifolds - Rev C.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMF- Bonded Manifolds -Rev G_scanned for TF.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP -

In [95]:
unstructerd_files=[]
for i in range(5):
    if pathes[i].endswith(".pdf"):
        strategy=shared.Strategy.VLM
    elif pathes[i].endswith(".doc") or pathes[i].endswith(".docx"):
        strategy=shared.Strategy.FAST
    el=get_elements_from_file(pathes[i],strategy)
    unstructerd_files.append(el)


INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1

In [100]:

for file in unstructerd_files:
    print(file)
    for element in file:
        print(element)
        
        
        



[{'type': 'UncategorizedText', 'element_id': '6b37173e9c5180e77bfe620773f2ef81', 'text': '', 'metadata': {'category_depth': 0, 'page_number': 1, 'text_as_html': '<div class="Page" data-page-number="1" id="7d1da0c1e4e74ab29dc0352a70537212" />', 'languages': ['eng'], 'filetype': 'application/pdf', 'partitioner_type': 'vlm_partition', 'filename': 'ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf'}}, {'type': 'Title', 'element_id': 'f04ab8375d52fc9874f380be7a6e2fd1', 'text': 'ArtPrompt: ASCII Art-based Jailbreak Attacks against Aligned LLMs', 'metadata': {'category_depth': 1, 'page_number': 1, 'parent_id': '7d1da0c1e4e74ab29dc0352a70537212', 'text_as_html': '<h1 class="Title" id="b2f3acf3a4a0427da98fe18d172379d0">ArtPrompt: ASCII Art-based Jailbreak Attacks against Aligned LLMs</h1>', 'languages': ['eng'], 'filetype': 'application/pdf', 'partitioner_type': 'vlm_partition', 'filename': 'ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf'}}, {'type': '

In [ ]:
import pandas as pd
cols=['type','element_id','text','metadata','filename','page_number','text_as_html','language','filetype','partition_type']
metadata_cols=['filename','page_number','text_as_html','language','filetype','partition_type']
df=pd.DataFrame(columns=cols)
def add_file_to_df(file: list, df: pd.DataFrame):
    for element in file:
        # Create a dictionary for the row
        row_dict = {
            'type': element['type'],
            'element_id': element['element_id'],
            'text': element['text'],
            'metadata': element['metadata']
        }
        
        # Extract each metadata column if it exists
        for meta_col in metadata_cols:
            if meta_col in element['metadata']:
                row_dict[meta_col] = element['metadata'][meta_col]
            else:
                row_dict[meta_col] = None  # Set to None if metadata field doesn't exist
                
        # Using concat instead of append (which is deprecated)
        df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)
    return df

for file in unstructerd_files:
    df=add_file_to_df(file,df)



In [133]:
len(df)

908

In [128]:
#count how many time an empty string appears in the text column
def check_if_element_already_exists(df, element):
    # Check if the element already exists in the DataFrame
    if element['metadata']['filename'] in df['filename'].unique().tolist():
        return True
    else:
        return False